In [0]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [102]:
!pip install tensorboardcolab

In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [103]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://4c452093.ngrok.io


In [0]:
GDRIVE_DIR = "/content/gdrive/My Drive"

In [0]:
training_examples_limit = None  # 'None' for training on all examples
test_examples_limit = None  # 'None' for testing on all examples
training_data_file = '/content/gdrive/My Drive/TIMIT/train.mfcccsv'
training_labels_file = '/content/gdrive/My Drive/TIMIT/train.targcsv'
test_data_file = '/content/gdrive/My Drive/TIMIT/test.mfcccsv'
test_labels_file = '/content/gdrive/My Drive/TIMIT/test.targcsv'

In [0]:
def read_training_test_data():
    # Read training data
    df_train = pd.read_csv(training_data_file, sep=',', nrows=training_examples_limit)
    np_train = df_train.values

    df_train_labels = pd.read_csv(training_labels_file, sep=',', nrows=training_examples_limit)
    train_labels = df_train_labels.values

    # Read test data
    df_test = pd.read_csv(test_data_file, sep=',', nrows=test_examples_limit)
    np_test = df_test.values

    df_test = pd.read_csv(test_labels_file, sep=',', nrows=test_examples_limit)
    test_labels = df_test.values

    return np_train, train_labels, np_test, test_labels

In [0]:
def extract_features():
    np_train, train_labels, np_test, test_labels = read_training_test_data()
    return (np_train, train_labels), (np_test, test_labels)

In [0]:
def neighbor_n_features(feature_vector, n):
    firsts = np.array([feature_vector[0], feature_vector[0]])
    lasts  = np.array([feature_vector[len(feature_vector)-1], feature_vector[len(feature_vector)-1]])
    copy_neighbors = np.append(firsts, feature_vector, axis=0)
    copy_neighbors = np.append(copy_neighbors, lasts, axis=0)
    neighbors = np.array([])
    for i, value in enumerate(feature_vector):
        if(len(neighbors)): #if it is not empty#
            neighbors = np.append(neighbors, np.array([copy_neighbors[i:i+n]]), axis=0)
        else: 
            neighbors = np.array([copy_neighbors[i:i+n]])
    #should return 3D array
    return neighbors

def extract_with_neighbor_features():
    np_train, train_labels, np_test, test_labels = read_training_test_data()
    neighbors_amount = 5
    neighbors_samples = neighbor_n_features(np_train, neighbors_amount)
    neighbors_test = neighbor_n_features(np_test, neighbors_amount)
    return (neighbors_samples, train_labels), (neighbors_test, test_labels)

In [0]:
def extract_with_derivatives(): 
    np_train, train_labels, np_test, test_labels = read_training_test_data()
    derivatives_training = get_dimensional_vector(np_train)
    derivatives_test     = get_dimensional_vector(np_test)
    dataframe_training = pd.DataFrame.from_records(derivatives_training)
    dataframe_test = pd.DataFrame.from_records(derivatives_test)
    dataframe_test.to_csv(test_derivative_data_file, encoding='utf-8', index=False)
    dataframe_training.to_csv(training_derivative_data_file, encoding='utf-8', index=False)
    np.set_printoptions(threshold=np.nan)
    return (derivatives_training, train_labels), (derivatives_test, test_labels)

def get_dimensional_vector(feature_vector):
    first = np.array([feature_vector[0]])
    last  = np.array([feature_vector[len(feature_vector)-1]])
    copy_neighbors = np.append(first, feature_vector, axis=0)
    copy_neighbors = np.append(copy_neighbors, last, axis=0)
    neighbors = np.array([])
    for i, value in enumerate(feature_vector):
        if(len(neighbors)): #if it is not empty#
            neighbors = np.append(neighbors, calculate_derivatives(copy_neighbors[i], copy_neighbors[i+1], copy_neighbors[i+2]), axis=0)
        else: 
            neighbors = calculate_derivatives(copy_neighbors[i], copy_neighbors[i+1], copy_neighbors[i+2])
    return neighbors

def calculate_derivatives(previous, current, following):
    delta = np.add(previous, following) / 2
    delta_delta = np.add(np.subtract(previous, 2*current), following)
    return np.array([np.append(current, np.append(delta, delta_delta))])

In [112]:
num_classes = 40

hidden_size = 1024
batch_size = 512
dropout_rate = 0.2
epochs = 20
activation_function = 'relu'
classificaton_function = 'softmax'
#optimizer = SGD(lr=0.1, momentum=0.0, decay=0.0, nesterov=False)
#optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#optimizer = RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0)
#optimizer = 'adam'
optimizer = RMSprop()

MODE = "simple"


if MODE == "simple":
    (x_train, y_train), (x_test, y_test) = extract_features()
elif MODE == "neighbors":
    (x_train, y_train), (x_test, y_test) = extract_with_neighbor_features()
elif MODE == "derivatives":
    (x_train, y_train), (x_test, y_test) = extract_with_derivatives()

    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1])
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1])

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

1422806 train samples
519938 test samples


In [113]:
model = Sequential()
if MODE == "neighbors":
    model.add(Dense(hidden_size, activation=act_function_hidden, input_shape=(x_train[0].shape[0], x_train[0].shape[1],)))
else:
    model.add(Dense(hidden_size, activation=act_function_hidden, input_shape=(x_train.shape[1],)))
#model.add(Dense(hidden_size, activation=act_function_hidden, input_shape=(13,)))
model.add(Dropout(dropout_rate))

if MODE == "neighbors":
    model.add(Flatten())
model.add(Dense(hidden_size, activation=activation_function))
model.add(Dropout(dropout_rate))
model.add(Dense(hidden_size, activation=activation_function))
model.add(Dropout(dropout_rate))

model.add(Dense(num_classes, activation=classificaton_function))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_148 (Dense)            (None, 1024)              14336     
_________________________________________________________________
dropout_121 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_149 (Dense)            (None, 1024)              1049600   
_________________________________________________________________
dropout_122 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_150 (Dense)            (None, 1024)              1049600   
_________________________________________________________________
dropout_123 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_151 (Dense)            (None, 40)                41000     
Total para

In [114]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

tensorBoardCallback = TensorBoard(log_dir='./logs/sequential', write_graph=True)
earlyStopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[earlyStopper, reduce_lr, TensorBoardColabCallback(tbc)])

score = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 1422806 samples, validate on 519938 samples
Epoch 1/20
1422806/1422806 [==============================] - 43s 30us/step - loss: 1.6183 - acc: 0.5153 - val_loss: 1.5068 - val_acc: 0.5363
Epoch 2/20
1422806/1422806 [==============================] - 42s 29us/step - loss: 1.5535 - acc: 0.5293 - val_loss: 1.5333 - val_acc: 0.5329
Epoch 3/20
1422806/1422806 [==============================] - 41s 29us/step - loss: 1.5600 - acc: 0.5293 - val_loss: 1.5176 - val_acc: 0.5352
Epoch 4/20
1422806/1422806 [==============================] - 42s 29us/step - loss: 1.5645 - acc: 0.5293 - val_loss: 1.5093 - val_acc: 0.5363
Test loss: 1.509314155100379
Test accuracy: 0.536285095530621
